# Tutorial 0: Before Starting
if you want to know about the relevant information and usage instructions about this library, please refer to the README.md or the README.en.md.

## Auxiliary parameter generation
Currently, we simulate the auxiliary parameters provided by a trusted third party using local files. If you want to generate these auxiliary parameters, please run```./debug/crypto/offline_provider.py```. Additionally, you can change the number of parameters generated according to your needs. **Note that don't forget to modify the running path to the root directory of the computation library before running**.
In this way, the parameters can be generated as follows: Beaver triples for multiplication, wrap keys for truncation and auxiliary parameters required by four different comparison methods: MSB Beaver triples used in the MSB (Most Significant Bit) method, DICF (Distributed Interval Containment Function) keys used in the DICF method, PPQCompareKey keys used in the PPQ method and SigmaCompareKey keys used in the SIGMA method.

In [1]:
from crypto.primitives.beaver.beaver_triples import BeaverTriples
from crypto.primitives.beaver.msb_triples import MSBTriples
from crypto.protocols.function_secret_sharing import *
from crypto.protocols.arithmetic_secret_sharing.truncate import Wrap

BeaverTriples.gen_and_save(100000, 2, 'TTP')
Wrap.gen_and_save(100000)
MSBTriples.gen_and_save(100000, 2, 'TTP')
PPQCompareKey.gen_and_save(100000)
DICFKey.gen_and_save(100000)
SigmaCompareKey.gen_and_save(100000)

In addition to these keys, other parameters are also required for some operations, such as the matrix beaver triples for matrix multiplication, which is related to the size of the matrix involved in the operation. Such parameters are usually related to actual operations, so it is hard to generate them in advance, and the generation scheme will be shown in subsequent tutorials.

## Configuration file
Here is an explanation of the configuration files. For convenience, we have put all the configuration files in the py file. The configuration files are divided into basic configuration```./config/base_configs.py``` and network configuration```./config/network_config.py```.
The network configuration is mainly about the configuration of the addresses, ports and other information of the two party in the security two-party calculation. It can be modified according to the actual situation, or it can be configured according to the actual situation when creating the participant object. As for how to configure the socket information of the participant, it will be introduced in the subsequent tutorial.
Now, let's have an insight into the basic configuration so that you can change the configuration to achieve different operations in the future.

#### base config
Main base config is as follows:
Our basic operations are on the ring. The bit number of the ring is represented by ```BIT_LEN```, and the corresponding size of the ring is ```2**BIT_LEN```. The range of values that can be represented is ```[-2 ** (BIT_LEN - 1), 2 ** (BIT_LEN - 1) - 1]```. The ring size supported is either 64 or 32. In the ./config/base_configs.py, there are several configurations related to BIT_LEN, such as the ring size (```RING_MAX```) and half-ring size (```HALF_RING```). For these configurations related to BIT_LEN, please do not modify them. What you only need to change is the value of BIT_LEN itself.

In [2]:
BIT_LEN = 64

All operations in our library support CPU and GPU computation. Therefore, you can choose the appropriate "device" based on your computer configuration: cpu, cuda(cuda:0, cuda:1).

In [3]:
DEVICE = 'cuda'

Below are several configurations used for number comparison in our library. We support several methods for comparing encrypted data, namely MSB, DICF, PPQ and SIGMA. You can switch between these methods by modifying the GE_TYPE configuration. We divide the comparison methods into two types: MSB-based and FSS-based. DICF, PPQ and SIGMA belong to the FSS based method. Through theory and experiment, it can be seen that the MSB method needs to use multiple communication, but the amount of computation is relatively small. The DICF method has a large amount of computation, but only one communication is needed, PPQ is an optimised DICF method that reduces the size of the key in the evaluation phase. SIGMA uses the same key size as PPQ but reduces the computational complexity of the comparison operation.
PRG_TYPE is the type of pseudorandom number generator. Currently, random numbers are required when using the FSS based comparison methods. We offer two options: TMT and AES. TMT is an exponential random number generator, while AES generates random numbers based on the AES encryption algorithm. When dealing with small amounts of data, the use of AES is considered to be more secure and efficient.

In [4]:
GE_TYPE = 'MSB'
PRG_TYPE = 'AES'

In our lib we provide three debug levels and call them level 0, 1, 2. In level 0, the auxiliary parameters change each time they are used. In the other two levels, the size of the auxiliary parameters is fixed. The difference between the two levels is that in level 1, the size of the auxiliary parameters is the same as the level 0, while in level 2, the size of the auxiliary parameters is fixed as one. If your computer cannot support the large number of auxiliary parameters (usually due to not having enough memory or cuda memory), you can use level 1 or 2 to simulate and obtain the running time and communication cost of the function. Note that the safety of the calculation is not guaranteed in these two levels.

In [5]:
DEBUG_LEVEL = 0
assert DEBUG_LEVEL in (0, 1, 2)

This section contains configuration information relevant to the numbers used in calculations. We categorise numbers as integers (int) and floats (float). You can change the DTYPE to suit your specific calculation requirements.
The SCALE represents the scale of the fractional part. For integer operations, there is no fractional part, so ``int_scale`` is always set to 1. For 64-bit floating point numbers, we recommend using 65536 (2 to the power of 16) as the scale value. For 32-bit floats, it is recommended that float_scale does not exceed 256.
You usually don't need to change the SCALE value, but if you do, you can only change the parts related to 65536 and 128.
Note that if the ``float_scale'' value exceeds the recommended range, there is a high probability that most calculations will fail.

In [6]:
DTYPE = 'float'
float_scale = 65536 if BIT_LEN == 64 else 128
int_scale = 1
SCALE = float_scale if DTYPE == 'float' else int_scale

Finally, there is the configuration related to storage paths. It is recommended not to make any changes to the default configuration. However, if you need to add a new file storage path, you can do it according to the actual situation.

In [7]:
base_path = f'./data/{BIT_LEN}'
model_file_path = base_path + '/neural_network/'

#### Some utils
In ```./common/utils/debug_utils.py```, we provide some tools to help you debug the code. For example, you can use the following code to check the time of a function:
```python
from common.utils import get_time
res = get_time(func, *args)
```
`get_time` will return the result of the function and print the time it takes to run the function. The parameters of `get_time` are the function to be tested and the parameters of the function.

IIn addition, we provide a function for counting the communication costs in secure multiparty computation tasks. You can use the following code to check the communication cost of a function:
```python
from common.utils import comm_count
res = comm_count(communicator, func, *args)
```
`comm_count` will return the result of the function and print the communication cost of the function. The parameters of `comm_count` are the parties in `model/mpc/semi_honest_party.py` or Communicator object in `common/network/communicator.py`, the function to be tested and the parameters of the function.

#### Note
If you find that some functions mentioned in this tutorial can not run, don't worry. It may be because the auxiliary parameters required for some functions are not generated or the auxiliary parameters are insufficient. You can refer to the tutorial and codes in the```./debug``` package to generate the auxiliary parameters required according to your needs, and distribute the calculation to multiple parties.